<a href="https://colab.research.google.com/github/Pranavh-2004/AI_Media_Detection-Synapse2.0/blob/main/objectcoords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
import zipfile
import os
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
zipref=zipfile.ZipFile("10_food_classes_all_data.zip")
zipref.extractall()
zipref.close()

--2024-04-19 20:54:01--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.10.207, 142.251.12.207, 172.217.194.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.10.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip’

10_food_classes_all 100%[===================>] 495.13M  22.4MB/s    in 25s     

2024-04-19 20:54:26 (20.2 MB/s) - ‘10_food_classes_all_data.zip’ saved [519183241/519183241]



In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


tf.random.set_seed(42)

# Define ImageDataGenerators for training and testing

train_datagen_augmented = ImageDataGenerator(
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    height_shift_range=0.2,
    width_shift_range=0.2

)

test_datagen = ImageDataGenerator(
    #rescale=1./255

)
train_datagen = ImageDataGenerator(
    #rescale=1./255

)

# Flow training images in batches using the generator
train_data_augmented = train_datagen_augmented.flow_from_directory(
    '10_food_classes_all_data/train/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=True
)

# Flow validation images in batches using the generator
test_data = test_datagen.flow_from_directory(
    '10_food_classes_all_data/test/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    seed=42
)
train_data=train_datagen.flow_from_directory(
    '10_food_classes_all_data/train/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=True
)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.
Found 7500 images belonging to 10 classes.


In [6]:
import tensorflow as tf
import numpy as np

base_model=tf.keras.applications.EfficientNetB4(include_top=False)
base_model.trainable=False #we are taking pre trained model so we shouldnt train it and disturb its accuracy

inputs=tf.keras.layers.Input(shape=(128,128,3))
x=base_model(inputs)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
outputs=tf.keras.layers.Dense(10,activation='softmax')(x)
model=tf.keras.models.Model(inputs,outputs)
model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               metrics=['accuracy'])


71686520/71686520 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 efficientnetb4 (Functional  (None, None, None, 1792   17673823  
 )                           )                                   
                                                                 
 global_average_pooling2d (  (None, 1792)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 10)                17930     
                                                                 
Total params: 17691753 (67.49 MB)
Trainable params: 17930 (70.04 KB)
Non-trainable params: 17673823 (67.42 MB)
_________________

In [7]:
model.fit(train_data_augmented,epochs=3,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)))

Epoch 1/3
235/235 [==============================] - 77s 252ms/step - loss: 1.2630 - accuracy: 0.6052 - val_loss: 0.6792 - val_accuracy: 0.7875
Epoch 2/3
235/235 [==============================] - 56s 239ms/step - loss: 0.9388 - accuracy: 0.6980 - val_loss: 0.6767 - val_accuracy: 0.8042
Epoch 3/3
235/235 [==============================] - 53s 225ms/step - loss: 0.8767 - accuracy: 0.7176 - val_loss: 0.5782 - val_accuracy: 0.8271


In [8]:
base_model.trainable=True
for i,layer in enumerate(base_model.layers[:-10]):
  layer.trainable=False

In [9]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),#new learning rate after changing layers=old learning rate/10
               metrics=['accuracy'])
model.fit(train_data_augmented,epochs=6,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)),initial_epoch=3)

Epoch 4/6
235/235 [==============================] - 76s 255ms/step - loss: 0.8664 - accuracy: 0.7171 - val_loss: 0.6233 - val_accuracy: 0.8000
Epoch 5/6
235/235 [==============================] - 55s 233ms/step - loss: 0.7746 - accuracy: 0.7456 - val_loss: 0.5537 - val_accuracy: 0.8125
Epoch 6/6
235/235 [==============================] - 54s 228ms/step - loss: 0.7129 - accuracy: 0.7620 - val_loss: 0.5278 - val_accuracy: 0.8292


In [20]:
# Perform inference on sample images from the training data
images, labels = next(train_data)  # Get one batch for inference
predictions = model.predict(images)

# Convert predictions to desired format
object_positions_list = []
for pred in predictions:
    object_positions = []
    for i in range(0, len(pred), 4):  # Assuming each object has 4 coordinates (a, b, c, d)
        if i + 4 <= len(pred):  # Check if there are enough values for unpacking
            a, b, c, d = pred[i:i+4]
            object_positions.append({'x': [[a, b], [c, d]]})
        else:
            break
    object_positions_list.append({'objects': object_positions})

print(object_positions_list)





1/1 [==============================] - 0s 35ms/step
[{'objects': [{'x': [[0.033731963, 0.0051096044], [0.022005234, 0.07274693]]}, {'x': [[0.00475481, 0.0014410185], [0.008701401, 0.0006028445]]}]}, {'objects': [{'x': [[2.918629e-05, 0.0007789364], [3.8041452e-07, 0.98972964]]}, {'x': [[7.968514e-05, 4.0096547e-06], [1.8771818e-05, 6.3805796e-06]]}]}, {'objects': [{'x': [[0.009953657, 0.00010998482], [0.0014655878, 0.008566509]]}, {'x': [[0.00012269623, 0.00022677687], [2.0408634e-05, 0.96875584]]}]}, {'objects': [{'x': [[0.00055645505, 3.8104907e-05], [0.0017220118, 8.031161e-05]]}, {'x': [[5.288905e-05, 0.99663407], [5.312984e-05, 0.0007759536]]}]}, {'objects': [{'x': [[0.015831172, 0.0006833043], [0.0015981463, 0.00039199868]]}, {'x': [[0.0003311118, 0.0001388244], [0.00093891314, 0.9769976]]}]}, {'objects': [{'x': [[0.0029412874, 0.0005573219], [0.00021024534, 0.0004007539]]}, {'x': [[0.000311024, 0.0013752559], [8.548732e-05, 4.678985e-06]]}]}, {'objects': [{'x': [[0.4002878, 0.02